This code is used for reading the original signals (ECG, PPG, ABP) from the CSV files, and extract features and labels from them. Then saving a dataset of those features & labels to be used for the train & test processes. 

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # for the progress bar :) 

## Helper Functions

In [8]:
def Process_Instant(Signals, Subset_Size = 125*3): # defulat Subset_Size to cover 3 seconds of data samples
    
    import numpy as np
    from scipy.signal import find_peaks
    import pandas as pd
    
    N_tot = len(Signals.index) # Total number of samples
    N_ps = Subset_Size # Num of points per subset
    N_ss = N_tot//N_ps # Number of subsets (each with N_ps points), discarding the last points < N_ps
    All_data=pd.DataFrame() # an empty DF to store the parameters we got from the processed data 
    fs=125 # used to convert PPTp from Number_of_samples to time (same as in main. If changed, pass it to this fn)
    
    for i in range(N_ss):  # USE find_peaks_cwt instead??
           
        # I found, imperically, that find_peaks works better for PPG compared with ECG & ABP
        # So, I am detecting PPG peaks fisrt, then using 0.8*its average distance to be the min spacing for ECG & ABP peaks!
        # Hopefully this may return more accurate PTT calculations
        
        
        # PPG peaks' locations
        PPG_peaks, _ = find_peaks(Signals['PPG'][i*N_ps:(i+1)*N_ps], distance=40)
        if len(PPG_peaks)<2: 
            print('PPG')
            continue
        
        New_dist = 0.8*np.mean(np.diff(PPG_peaks))
        
        # PPG min' locations
        PPG_min, _ = find_peaks(-1*Signals['PPG'][i*N_ps:(i+1)*N_ps], distance=New_dist)
        if len(PPG_min)<2: 
            print('PPG')
            continue
        
        # ECG peaks' locations & MAP 
        ECG_peaks, _ = find_peaks(Signals['ECG'][i*N_ps:(i+1)*N_ps], distance=New_dist) 
        MAP = np.mean(Signals['ABP'].iloc[i*N_ps + np.arange(ECG_peaks[0], ECG_peaks[-1]+1)].values) #### VALUES not indicies
        if len(ECG_peaks)<2: 
            print('ECG')
            continue
        
        
        if PPG_peaks[-1]<ECG_peaks[0]: 
            print('ECG lagging too much')
            continue
        
        # PPT Calculations:
        # The 1st PPG peak has to lag the 1st ECG peak, if not then drop it and consider the next one.
        Lp, Le, Lm = len(PPG_peaks), len(ECG_peaks), len(PPG_min)
        min_pe = min(Lp, Le, Lm)
        Lpe = min(Lp,Le)
        Lme = min(Lm,Le)
        Lmp = min(Lp,Lm)
        
        if (PPG_peaks[0]>ECG_peaks[0]) and (PPG_min[0]>PPG_peaks[0]): # may add more conditions here for ACCEPTABLE PTT values
            PTT_vec = PPG_peaks[:Lpe] - ECG_peaks[:Lpe]
            PTT_min_vec = PPG_min[:Lme] - ECG_peaks[:Lme]
            PTT_h_vec = Signals['PPG'].iloc[i*N_ps + PPG_peaks[:Lmp]].values - Signals['PPG'].iloc[i*N_ps + PPG_min[:Lmp]].values #### VALUES not indicies
            
        else: # start from the 2nd PPG peak all the way to one before the last ECG peak
            s = 1
            while (PPG_peaks[s]<ECG_peaks[0]) and (s<=Lp-1):
                s+=1
            ind_max_ECG = min(Lp-s,Le) # common last elem index between max & ECG vectors
            PTT_vec = PPG_peaks[s:s+ind_max_ECG] - ECG_peaks[:ind_max_ECG] # PATp
            
            # Now making sure that min's follows max's
            r = 0
            while (PPG_min[r]<PPG_peaks[s]) and (r<=Lm-1):
                r+=1
            ind_m_ECG = min(Lm-r,Le) # common last elem index between min & ECG vectors
            PTT_min_vec = PPG_min[r:r+ind_m_ECG] - ECG_peaks[:ind_m_ECG] # PATf
            ind_h_min = min(Lp-s, Lm-r) # common last elem index between min & max vectors
            PTT_h_vec = Signals['PPG'].iloc[i*N_ps + PPG_peaks[s:s+ind_h_min]].values - Signals['PPG'].iloc[i*N_ps + PPG_min[r:r+ind_h_min]].values # PPGmax - PPGmin (i.e. PPG height)
        
        PTT_vec_p = [x for x in PTT_vec if x>0] # forcing positive values
        PTT_min_vec_p = [x for x in PTT_min_vec if x>0]
        PTT_h_vec_p = [x for x in PTT_h_vec if x>0]
        
        PTT_avg = np.mean(PTT_vec_p)
        PTT_min_avg = np.mean(PTT_min_vec_p)
        PTT_h_avg = np.mean(PTT_h_vec_p)
        
        
        
        # BP Min & Max Peaks' (DBP & SBP) locations
        BPmax_peaks, _ = find_peaks(Signals['ABP'][i*N_ps:(i+1)*N_ps], distance=New_dist) # indices of SBP
        BPmin_peaks, _ = find_peaks(-1*Signals['ABP'][i*N_ps:(i+1)*N_ps], distance=New_dist) # indices of DBP
        
        
            # Calculate HR by averaging distances from ECG, PPG, BPmax, BPmin
        HR_ECG = np.mean(np.diff(ECG_peaks))
        HR_PPG = np.mean(np.diff(PPG_peaks))
        HR_PPG_min = np.mean(np.diff(PPG_min))
        HR_SBP = np.mean(np.diff(BPmax_peaks))
        HR_DBP = np.mean(np.diff(BPmin_peaks))
        HR_avg = 125*60/((HR_ECG + HR_PPG + HR_SBP + HR_DBP + HR_PPG_min)/5) # HR per minute
        ############################ 125*60/(mean(diff))
        
        # Creating DF's of SBP, DBP, and HR along with the time axis to be merged with the Subset DF
        SBP_avg = np.mean(Signals.iloc[i*N_ps + BPmax_peaks].loc[:,['ABP']])
        DBP_avg = np.mean(Signals.iloc[i*N_ps + BPmin_peaks].loc[:,['ABP']]) 
        
        
        # Append log_PPT, log-scale is more convinient for estimating BP (inspired from an approx eqn)
        Subset_DF = pd.DataFrame({'PTT':np.log(PTT_avg/fs), 
                                  'PTTm':np.log(PTT_min_avg/fs), 
                                  'PTTh':PTT_h_avg,
                                  'SBP':SBP_avg,
                                  'DBP':DBP_avg,
                                  'MAP':MAP,
                                  'HR':HR_avg})
         
        
        # Finally, append the DF of the current subset to the whole DF for all patients (may add more numerics, ID, ... later)
        All_data=All_data.append(Subset_DF, ignore_index=True)
    return All_data
    

## Loading data and calculating indices 

In [2]:
# for single csv file of 1,000 instants, use this directory & change file name as needed
#Data_Dir = "C:/1_Work/FANOS_lab/Reduced_Processed_Dataset_UCI_Kauchee/Generated_CSVs_from_Reduced_Dataset/12_Files_1000_instant_each/"
#File_name = "1.csv" # files 1 - 12, each with 1,000 instants

# To load all data from a single file: 
Data_Dir = "C:/1_Work/FANOS_lab/Reduced_Processed_Dataset_UCI_Kauchee/Generated_CSVs_from_Reduced_Dataset/Single_File/"
File_name = "All_Signals.csv" # All instants, 12,000, in a single csv file

# Now read data header-less then give the col names
Signals = pd.read_csv(Data_Dir + File_name, header=None) #.reset_index()
Signals.columns = ['PPG','ABP','ECG'] # adding columns' names

# Reading the instants' lengthes to be used for calculating indices
Data_Indices = pd.read_csv(Data_Dir+"Instants_Length.csv", header=None)
Data_Indices.columns = ['Num_Samples']

# Calculating Instant Indices, Start & End
End_indices = np.cumsum(Data_Indices["Num_Samples"])
Start_indices = np.append(0, End_indices) 

# Sampling frequency
Fs=125

FileNotFoundError: File b'C:/1_Work/FANOS_lab/Reduced_Processed_Dataset_UCI_Kauchee/Generated_CSVs_from_Reduced_Dataset/Single_File/All_Signals.csv' does not exist

## Feature Extraction: Converting signals into PTT, PTTm, PTTh, HR, SBP, DBP, MAP values

In [2]:
# Looping over the instants, processing them one by one, storing processed data (PTT, SBP, DBP) in another dataframe
All_Instants_Data = pd.DataFrame() 

# Which instants would you like to explore?
iStart, iEnd = 1, 12000 # 1:12,000 
Instants_ID = np.arange(iStart - 1, iEnd)
Subset_Size = 8*125 # how many seconds to take at a time

for ID in tqdm(Instants_ID):  
    Instant_DF = Process_Instant(Signals.iloc[Start_indices[ID]:End_indices[ID]+1], Subset_Size)  # Processing data for 1 instant
    All_Instants_Data = All_Instants_Data.append(Instant_DF, ignore_index=True) # Appending to the whole processed dataset

  0%|                                                                                        | 0/12000 [00:00<?, ?it/s]


NameError: name 'Process_Instant' is not defined

### Check and save the generated dataset

In [24]:
All_Instants_Data.describe() # check on the dataset

,PTTh,PTTm,PTT,HR,SBP,DBP,MAP
count,224017.000000,224017.000000,224017.000000,224017.000000,224017.000000,224017.000000,224017.000000
mean,1.502436,1.125356,0.575596,92.394599,133.576023,71.387614,93.667467
std,0.471776,0.467996,0.328781,15.733808,20.316135,9.771469,11.612897
min,0.132734,0.030500,0.008000,54.415423,80.019000,60.000000,66.646760
25%,1.391544,0.924800,0.432667,80.456582,118.808333,64.047526,85.024067
50%,1.673225,1.120000,0.612571,91.519219,132.973333,68.917933,91.924155
75%,1.827060,1.282667,0.664615,102.120974,148.020000,76.167333,100.996230
max,2.608299,4.086000,2.256800,155.795596,180.000000,129.955385,148.771474


In [14]:
All_Instants_Data.to_csv('Extracted_Instants_Parameters_8secWindow_PTTm_PTTh_MAP_210621.csv') # Saving Dataset